## HW4

### Question 1

In [4]:
from LOB_GAN_training_raw import train_lob_gan

stock_list = ["0050", "0056", "2330"]

lr_config = {
    "0050": {"lr_g": 0.004, "lr_d": 0.0008},
    "0056": {"lr_g": 0.006, "lr_d": 0.0005},
    "2330": {"lr_g": 0.0045, "lr_d": 0.0012},
}

batch_config = {
    "0050": 50,
    "0056": 50,
    "2330": 50,   
}

results = {}

for stock in stock_list:
    print(f"===== Training GAN for {stock} =====")
    cfg = lr_config[stock]
    bs = batch_config[stock]
    res = train_lob_gan(
        stock=stock,
        lr_g=cfg["lr_g"],
        lr_d=cfg["lr_d"],
        batch_size=bs,
        seed=307,
    )
    results[stock] = res

===== Training GAN for 0050 =====
Raw data loading and processing 0050
Data 1 for 0050 loaded.
Data 2 for 0050 loaded.
Data 3 for 0050 loaded.
Minutely data generated.
Start training 0050: epochs=200, batch_size=50, lr_g=0.004, lr_d=0.0008
[Epoch 1/200][Batch 1/1][D train loss: 0.250004][G train loss: 1.294996]
[Epoch 1/200][Batch 1/1][D eval loss: 0.250009][G eval loss: 1.117348]
[Epoch 2/200][Batch 1/1][D train loss: 0.250008][G train loss: 1.269022]
[Epoch 2/200][Batch 1/1][D eval loss: 0.249997][G eval loss: 1.107848]
[Epoch 3/200][Batch 1/1][D train loss: 0.249995][G train loss: 1.259430]
[Epoch 3/200][Batch 1/1][D eval loss: 0.249987][G eval loss: 1.107934]
[Epoch 4/200][Batch 1/1][D train loss: 0.249983][G train loss: 1.259470]
[Epoch 4/200][Batch 1/1][D eval loss: 0.249980][G eval loss: 1.107752]
[Epoch 5/200][Batch 1/1][D train loss: 0.249973][G train loss: 1.259352]
[Epoch 5/200][Batch 1/1][D eval loss: 0.249968][G eval loss: 1.106480]
[Epoch 6/200][Batch 1/1][D train loss: 0

In [ ]:
from LOB_GAN_training_raw import train_lob_gan

# 想要搜索的股票
stock_list = ["0050", "0056", "2330"]

# 网格搜索的学习率候选
lr_g_candidates = [0.0040, 0.0045, 0.005,0.006]
lr_d_candidates = [0.0005,0.0008, 0.0010, 0.0012]

batch_size = 50
seed = 307

# 用来记录所有结果： (stock, lr_g, lr_d) -> dict(...)
all_results = {}

# 用来记录每只股票的最优参数
best_config = {}
best_score = {}

for stock in stock_list:
    print(f"\n========== Grid search for {stock} ==========")
    best_config[stock] = None
    best_score[stock] = float("inf")

    for lr_g in lr_g_candidates:
        for lr_d in lr_d_candidates:
            print(f"\n----- {stock}: lr_g={lr_g}, lr_d={lr_d} -----")
            res = train_lob_gan(
                stock=stock,
                lr_g=lr_g,
                lr_d=lr_d,
                batch_size=batch_size,
                seed=seed,
            )

            if res is None:
                print(f"[WARN] {stock} lr_g={lr_g} lr_d={lr_d} 返回 None，跳过评分。")
                continue

            eval_g = res["eval_g_loss"]
            eval_d = res["eval_d_loss"]
            if len(eval_g) == 0 or len(eval_d) == 0:
                print(f"[WARN] {stock} lr_g={lr_g} lr_d={lr_d} 没有 eval loss，跳过。")
                continue

            score = eval_g[-1] + eval_d[-1]
            print(f"[INFO] {stock} lr_g={lr_g} lr_d={lr_d} -> score={score:.6f}")

            all_results[(stock, lr_g, lr_d)] = {
                "res": res,
                "score": score,
            }

            if score < best_score[stock]:
                best_score[stock] = score
                best_config[stock] = (lr_g, lr_d)

    print(f"\n>>> Best config for {stock}: "
          f"lr_g={best_config[stock][0]}, lr_d={best_config[stock][1]}, "
          f"score={best_score[stock]:.6f}")

print("\n========== Grid search finished ==========")
for stock in stock_list:
    if best_config[stock] is not None:
        print(f"{stock}: best lr_g={best_config[stock][0]}, "
              f"lr_d={best_config[stock][1]}, score={best_score[stock]:.6f}")
    else:
        print(f"{stock}: no valid config found.")



========== Grid search for 0050 ==========

----- 0050: lr_g=0.004, lr_d=0.0005 -----
Raw data loading and processing 0050
Data 1 for 0050 loaded.
Data 2 for 0050 loaded.
Data 3 for 0050 loaded.
Minutely data generated.
Start training 0050: epochs=200, batch_size=50, lr_g=0.004, lr_d=0.0005
[Epoch 1/200][Batch 1/1][D train loss: 0.250004][G train loss: 1.294996]
[Epoch 1/200][Batch 1/1][D eval loss: 0.249998][G eval loss: 1.119275]
[Epoch 2/200][Batch 1/1][D train loss: 0.249998][G train loss: 1.270949]
[Epoch 2/200][Batch 1/1][D eval loss: 0.249995][G eval loss: 1.108129]
[Epoch 3/200][Batch 1/1][D train loss: 0.249994][G train loss: 1.259711]
[Epoch 3/200][Batch 1/1][D eval loss: 0.249988][G eval loss: 1.106901]
[Epoch 4/200][Batch 1/1][D train loss: 0.249986][G train loss: 1.258437]
[Epoch 4/200][Batch 1/1][D eval loss: 0.249982][G eval loss: 1.106161]
[Epoch 5/200][Batch 1/1][D train loss: 0.249977][G train loss: 1.257762]
[Epoch 5/200][Batch 1/1][D eval loss: 0.249976][G eval los

In [1]:
from LOB_GAN_training_adjusted import train_lob_gan

stock_list = ["0050", "0056", "2330"]

lr_config = {
    "0050": {"lr_g": 0.004, "lr_d": 0.0008},
    "0056": {"lr_g": 0.006, "lr_d": 0.0005},
    "2330": {"lr_g": 0.0045, "lr_d": 0.0012},
}


batch_config = {
    "0050": 50,
    "0056": 50,
    "2330": 50,
}

results_adjusted = {}

for stock in stock_list:
    print(f"===== Training ADJUSTED GAN for {stock} =====")
    cfg = lr_config[stock]
    bs = batch_config[stock]
    res = train_lob_gan(
        stock=stock,
        lr_g=cfg["lr_g"],
        lr_d=cfg["lr_d"],
        batch_size=bs,
        seed=307,
    )
    results_adjusted[stock] = res


===== Training ADJUSTED GAN for 0050 =====
Raw data loading and processing 0050
Data 1 for 0050 loaded.
Data 2 for 0050 loaded.
Data 3 for 0050 loaded.
Minutely data generated.
Start training 0050: epochs=200, batch_size=50, lr_g=0.004, lr_d=0.0008
[Epoch 1/200][Batch 1/1][D train loss: 0.250004][G train loss: 1.297082]
[Epoch 1/200][Batch 1/1][D eval loss: 0.250009][G eval loss: 1.118483]
[Epoch 2/200][Batch 1/1][D train loss: 0.250008][G train loss: 1.270163]
[Epoch 2/200][Batch 1/1][D eval loss: 0.249997][G eval loss: 1.108594]
[Epoch 3/200][Batch 1/1][D train loss: 0.249995][G train loss: 1.260239]
[Epoch 3/200][Batch 1/1][D eval loss: 0.249986][G eval loss: 1.108031]
[Epoch 4/200][Batch 1/1][D train loss: 0.249982][G train loss: 1.259625]
[Epoch 4/200][Batch 1/1][D eval loss: 0.249979][G eval loss: 1.107663]
[Epoch 5/200][Batch 1/1][D train loss: 0.249973][G train loss: 1.259283]
[Epoch 5/200][Batch 1/1][D eval loss: 0.249967][G eval loss: 1.106205]
[Epoch 6/200][Batch 1/1][D trai

### Question 2